# Fichier: LECTURE ET ANALYSE DES DATASETS
### Importation des librairies

In [ ]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline.offline import _plot_html
import plotly.graph_objs as go
print (__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)

from mpl_toolkits import mplot3d
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib notebook
#%matplotlib inline

from utils import series_supp as ss
from utils import data_factory as df
from utils import plot
from utils import statics_func as sf

In [ ]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)
factory = df.DataFactory(cwd)
ploter = plot.Plot(None)

RG24 = ss.SeriesSupp(cwd, factory, "RG24")
RG1 = ss.SeriesSupp(cwd, factory, "RG1")
GW = ss.SeriesSupp(cwd, factory, "GW")

### Importation des datasets depuis les fichiers de stockage csv

In [ ]:
GW.import_dataset()
RG24.import_dataset()

#### Parametres du plot
Année et mois

In [ ]:
#for k,v in GW.tmp_dataset.items():
#    if len(v) > 168 or len(v) < 130:
#        GW.tmp_dataset[k] = [0]

In [ ]:
#for k,v in GW.tmp_dataset.items():
#    print(str(k)+" "+str(len(v)))

In [ ]:
ploter.change_mode(3)
years, months = [2015], [3]

GW.reset_dataset()
#RG24.reset_dataset()

GW.years = years
#GW.months = months
GW.reset_months()
#GW.reset_years()

#RG24.years = years
#RG24.months = months
#RG24.reset_months()
#RG24.reset_years()

GW.split_data_years()
GW.split_data_months()
#GW.split_data_weeks()

#RG24.split_data_years()
#RG24.split_data_months()
#RG24.split_data_weeks()

#RG24.dict_round()
#GW.dict_round()

#GW.dict_norm()
#RG24.dict_norm()

#### Plot RG

In [ ]:
#ploter.plot_scatter(RG24.get_data())

#### Plot GW

In [ ]:
ploter.change_mode(3)

In [ ]:
for k,v in GW.tmp_dataset.items():
    print(k)

In [ ]:
 cap = "GW_203_2015_09"

In [ ]:
ploter.plot_scatter_by_capteur(GW.get_data(), [cap])

# Tests: Auto-correlation, Stationarité

In [ ]:
import statsmodels.api as sm
import pandas as pd

In [ ]:
capV = GW.get_data()[cap]
capV = capV.set_index("Date")
w = 24

In [ ]:
capV.head(2)

In [ ]:
#sf.test_stationarity(capV)

In [ ]:
fig = plt.figure(figsize=(14,7))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(capV.values.squeeze(), lags=30, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(capV, lags=30, ax=ax2)

In [ ]:
moving_avg = capV.rolling(window=w,center=False).mean()
fig = plt.figure(figsize=(14,7))
plt.plot(capV)
plt.plot(moving_avg, color='red')

In [ ]:
capV_moving_avg_diff = capV - moving_avg
capV_moving_avg_diff.dropna(inplace=True)
sf.test_stationarity(capV_moving_avg_diff, w)

In [ ]:
#fig = plt.figure(figsize=(14,7))
#ax1 = fig.add_subplot(211)
#fig = sm.graphics.tsa.plot_acf(capV_moving_avg_diff.values.squeeze(), lags=24, ax=ax1)
#ax2 = fig.add_subplot(212)
#fig = sm.graphics.tsa.plot_pacf(capV_moving_avg_diff, lags=24, ax=ax2)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(capV, freq=w)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

fig = plt.figure(figsize=(14,8))
plt.subplot(411)
plt.plot(capV, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.show()